In [21]:
import os
import numpy as np
import torch
import pickle
import Models.Get_data as Gd
from torch.utils.data import DataLoader, Dataset
import scipy
from sklearn.decomposition import PCA

In [22]:
latent_space = 25
window_size = 10
step_size = 3

directory = '/home/wl4023/data/Sibo_22Mar2024'
folders = [os.path.join(directory, f, 'hessian_') for f in os.listdir(directory) if f.startswith('case_')]

In [23]:
with open('/home/wl4023/github_repos/IRP/preprocessor.pkl', 'rb') as f:
    preprocessor = pickle.load(f)
print(preprocessor.mean, preprocessor.std)

6.95707893371582 9.811119079589844


In [24]:
save_path = f'/home/wl4023/github_repos/IRP/predict_result_compare/Latent_space_{latent_space}/'
os.makedirs(save_path, exist_ok=True)

# PCA

In [25]:
dataset, length = Gd.get_all_nodes(folders[:-1])
dataset = dataset[:, :, 0]

pca = PCA(n_components=latent_space)
pca.fit(preprocessor.transform(dataset))

PCA(n_components=25)

In [26]:
class PCA_Dataset(Dataset):
    def __init__(self, x, y, original_x, original_y):
        self.x = x
        self.y = y
        self.original_x = original_x
        self.original_y = original_y
    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx], self.original_x[idx], self.original_y[idx]

In [27]:
dataset_1, length = Gd.get_all_nodes(folders[-1:])
dataset_1 = dataset_1[:, :, 0]
test_pca_data = pca.transform(preprocessor.transform(dataset_1))
test_pca_data = torch.tensor(test_pca_data, dtype=torch.float32)
test_pca_data = Gd.segment_data_torch(test_pca_data, length, window_size, step_size)
print(test_pca_data.shape)


dataset_1 = torch.tensor(dataset_1, dtype=torch.float32)
test_origin_dataset = Gd.segment_data_torch(dataset_1, length, window_size, step_size)

test_data = PCA_Dataset(test_pca_data[:, :5, :], test_pca_data[:, -5:, :], test_origin_dataset[:, :5, :], test_origin_dataset[:, -5:, :])
test_loader = DataLoader(test_data, 1)

torch.Size([31, 10, 25])


In [28]:
from Models.LSTM import LSTM_seq2seq

PCA_LSTM_seq2seq = LSTM_seq2seq(latent_space, 100)
state_dict = torch.load(f'/home/wl4023/github_repos/IRP/predict result/Latent space {latent_space}/PCA_LSTM.pth')

PCA_LSTM_seq2seq.load_state_dict(state_dict)

<All keys matched successfully>

In [29]:
pca_lstm_recon_predict_list = []

with torch.no_grad():
    test_loss = 0
    test_metric = 0
    for x, y, _, original_y in test_loader:
        input = x
        label = y
        predict = PCA_LSTM_seq2seq(input, 5)

        recon_predict=[]
        for i in range(len(x)):
            temp = preprocessor.inverse_transform(pca.inverse_transform(predict[i].cpu().detach().numpy()))
            recon_predict.append(torch.tensor(temp, dtype=torch.float32))

        recon_predict = torch.stack(recon_predict, dim=0)
        pca_lstm_recon_predict_list.append(recon_predict)


pca_lstm_recon_predict_list = torch.cat(pca_lstm_recon_predict_list).numpy()

np.save(save_path+'PCA_LSTM_predict.npy', pca_lstm_recon_predict_list)

print(pca_lstm_recon_predict_list.shape)

(31, 5, 97149)


# CAE_LSTM

In [30]:
dataset, length = Gd.get_all_nodes(folders[-1:])
dataset = np.expand_dims(dataset[:, :, 0], axis=1)

dataset = torch.tensor(dataset, dtype=torch.float32)
dataset = Gd.segment_data_torch(dataset, length, window_size, step_size)

test_data = Gd.MyDataset(dataset[:, 0:5, :, :], dataset[:, -5:, :, :])
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [31]:
from Models.CAE_LSTM import CAE_LSTM_seq2seq
model = CAE_LSTM_seq2seq(latent_space, 100)
state_dict = torch.load(f'/home/wl4023/github_repos/IRP/predict result/Latent space {latent_space}/CAE_LSTM.pth')

model.load_state_dict(state_dict)

<All keys matched successfully>

In [32]:
CAE_LSTM_recon_predict_list=[]

model.eval()
with torch.no_grad():
    test_loss = 0
    test_metric = 0
    for x, y in test_loader:
        input = preprocessor.transform(x)
        predict = model(input, 5)

        recon_predict = preprocessor.inverse_transform(predict)

        CAE_LSTM_recon_predict_list.append(recon_predict.cpu())


CAE_LSTM_recon_predict_list = torch.cat(CAE_LSTM_recon_predict_list, dim=0).numpy()
np.save(save_path+'CAE_LSTM_predict.npy', CAE_LSTM_recon_predict_list)

print(CAE_LSTM_recon_predict_list.shape)

(31, 5, 1, 97149)


# GAE_LSTM

In [33]:
pos = torch.tensor(np.load(directory+"/case_0/hessian_/xy_coords.npy"), dtype=torch.float32)

sparse_graph = scipy.sparse.load_npz(directory+'/new_sparse_matrix.npz')
indices = np.array(sparse_graph.nonzero())
values = sparse_graph.data
shape = sparse_graph.shape

# sparse edge tensor
edge_tensor = torch.sparse_coo_tensor(indices, values, shape, dtype=torch.float).coalesce()
edge_index = edge_tensor.indices()
edge_weight = edge_tensor.values()
edge_weight = edge_weight.unsqueeze(1)

In [34]:
dataset_1, length = Gd.get_all_nodes(folders[-1:])
dataset_1 = np.expand_dims(dataset_1[:, :, 0], axis=2)

dataset_1 = torch.tensor(dataset_1, dtype=torch.float32)
dataset_1 = Gd.segment_data_torch(dataset_1, length, window_size, step_size)

gae_lstm_test_set = Gd.MyDataset(dataset_1[:, 0:5, :, :], dataset_1[:, -5:, :, :])

In [35]:
clusters = torch.load(f'/home/wl4023/github_repos/IRP/result/Latent space {latent_space}/clusters.pt')
centroids = torch.load(f'/home/wl4023/github_repos/IRP/result/Latent space {latent_space}/centroids.pt')

In [36]:
from Models.GAE_LSTM import GAE_LSTM_seq2seq

gae_lstm_seq2seq = GAE_LSTM_seq2seq(latent_space=latent_space,
                                    hidden_size=100,
                                    input_node_channel=1,
                                    output_node_channel=1,
                                    num_mp_layers=[2, 2, 2],
                                    clusters=clusters,
                                    centroids=centroids,
                                    hidden_channels=16,
                                    n_mlp_mp=3)

state_dict = torch.load(f'/home/wl4023/github_repos/IRP/GAE_LSTM_result/Latent_space_{latent_space}/GAE_LSTM.pth')

gae_lstm_seq2seq.load_state_dict(state_dict)

<All keys matched successfully>

In [37]:
GAE_LSTM_recon_predict_list = []
with torch.no_grad():
    test_loss = 0
    test_metric = 0
    for x, y in gae_lstm_test_set:
        input = preprocessor.transform(x)
        predict = gae_lstm_seq2seq(input, 5, edge_index, edge_weight, pos)

        recon_predict = preprocessor.inverse_transform(predict.cpu())

        GAE_LSTM_recon_predict_list.append(recon_predict)

GAE_LSTM_recon_predict_list = torch.stack(GAE_LSTM_recon_predict_list, dim=0).numpy()

np.save(save_path+'GAE_LSTM_predict.npy', GAE_LSTM_recon_predict_list)
print(GAE_LSTM_recon_predict_list.shape)

(31, 5, 97149, 1)


# GAE_GConvLSTM

In [38]:
dataset_1, length = Gd.get_all_nodes(folders[-1:])
dataset_1 = np.expand_dims(dataset_1[:, :, 0], axis=2)

dataset_1 = torch.tensor(dataset_1, dtype=torch.float32)
dataset_1 = Gd.segment_data_torch(dataset_1, length, window_size, step_size)

GAE_GConvLSTM_test_set = Gd.MyDataset(dataset_1[:, 0:5, :, :], dataset_1[:, -5:, :, :])

In [39]:
from Models.GAE_GConvLSTM import GAE_GConvLSTM_seq2seq

gae_gconvlstm_seq2seq = GAE_GConvLSTM_seq2seq(latent_space=latent_space,
                                              hidden_channel_lstm=1,
                                              input_node_channel=1,
                                              output_node_channel=1,
                                              num_mp_layers=[2, 2, 2],
                                              clusters=clusters,
                                              centroids=centroids,
                                              hidden_channels=8,
                                              n_mlp_mp=3,)

state_dict = torch.load(f'/home/wl4023/github_repos/IRP/GAE_GConvLSTM_result/Latent_space {latent_space}/GConvLSTM.pth')

gae_gconvlstm_seq2seq.load_state_dict(state_dict)

<All keys matched successfully>

In [40]:
GAE_GConvLSTM_recon_predict_list = []
with torch.no_grad():
    test_loss = 0
    test_metric = 0
    for x, y in GAE_GConvLSTM_test_set:
        input = preprocessor.transform(x)
        predict = gae_gconvlstm_seq2seq(input, 5, edge_index, edge_weight, pos)

        recon_predict = preprocessor.inverse_transform(predict.cpu())

        GAE_GConvLSTM_recon_predict_list.append(recon_predict)

GAE_GConvLSTM_recon_predict_list = torch.stack(GAE_GConvLSTM_recon_predict_list, dim=0).numpy()


np.save(save_path+'GAE_GConvLSTM_predict.npy', GAE_GConvLSTM_recon_predict_list)
print(GAE_GConvLSTM_recon_predict_list.shape)

(31, 5, 97149, 1)
